In [96]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import pickle
import seaborn as sns

In [81]:
pd.set_option('display.max_columns', 100)

# Scrape data

In [82]:
def getAndParseUrl(url):
    results = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(results.text, 'html.parser')
    return soup

In [152]:
# this url is for all players between 2014 and 2019 with a PER >= 0
url1 = 'https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=per_game&per_minute_base=36&per_poss_base=100&lg_id=NBA&is_playoffs=N&year_min=2015&year_max=2019&franch_id=&season_start=1&season_end=-1&age_min=0&age_max=99&shoot_hand=&height_min=0&height_max=99&birth_country_is=Y&birth_country=&birth_state=&college_id=&draft_year=&is_active=&debut_yr_nba_start=&debut_yr_nba_end=&is_hof=&is_as=&as_comp=gt&as_val=0&award=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&qual=&c1stat=per&c1comp=gt&c1val=0&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&c5stat=&c5comp=&c6mult=&c6stat=&order_by=season&order_by_asc=&offset='

In [153]:
# this url is for all players between 2014 and 2019 with a PER <=0
url2 = 'https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=per_game&per_minute_base=36&per_poss_base=100&season_start=1&season_end=-1&lg_id=NBA&age_min=0&age_max=99&is_playoffs=N&height_min=0&height_max=99&year_min=2015&year_max=2019&birth_country_is=Y&as_comp=gt&as_val=0&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=per&c1comp=lt&c1val=0&order_by=season&order_by_asc=&offset='

In [441]:
def get_players(url): # function to grab every player and his stats from the page as a dictionary, and store as list
    
    # player dictionaries container
    players = []
    
    # loop through each page(for this website, each page is accessed by 'offset={some multiple of 100}')
    # last page is offset=2500
    for i in tqdm(range(0,3000,100)):
        url = url+str(i)
        soup = getAndParseUrl(url)
        
        # select each td tag under tr tag: each td tag in player_soup is a player's stat, all stored as a list
        player_soup = soup.select('tr td')
        
        # create slicing points to group the 33 elements(html tags/columns) into single list(player)
        j=0
        k=32
        
        # there are a 100 players per page, 31 stats per player == 3100 elements in player_soup (except last page)
        # len(player_soup) gives the number of stats on each page
        # divide that by 31 to get number of players per page
        for i in range(int(len(player_soup)/32)):
            
            # each player container (player with stats)
            each_player = {}
            
            # p is a single player with stats sliced from player_soup
            p = player_soup[j:k]
            
            #loop through p to create dictionary of each player
            for stat in p:
                # set each stat equal to the tag text (i.e. pts_per_g = 15.4)
                each_player[stat.attrs['data-stat']] = stat.text
            
            # add each player dictionary to the players container
            players.append(each_player)
            
            # move slicer to next player
            j+=32
            k+=32

        next_pg_checker = soup.select('p a[href]')
        if 'Next page' not in [c.text for c in next_pg_checker]:
            break
        else: continue
        
    return players

In [135]:
players=get_players(url1)

100%|██████████| 26/26 [00:55<00:00,  2.04s/it]


In [155]:
# check to see if all player dicts were created
len(players)

# THEY WERE

2473

In [156]:
players.extend(get_players(url2))

  0%|          | 0/26 [00:00<?, ?it/s]


In [159]:
# save the players as a pickle
with open('players.pickle', 'wb') as outfile:
    pickle.dump(players, outfile)

outfile.close()

# Setting up the dataframe

## NBA dataframe

In [160]:
# make a dataframe of all players and stats
df = pd.DataFrame(players, columns = players[0].keys())

##### change dtypes for columns with numerical data from str to int or float


In [161]:
cols = list(players[0].keys())

In [162]:
df.age = df.age.astype(int)

In [168]:
df.per = df.per.astype(float)

In [174]:
for i in range(6,len(cols)):
    df[cols[i]] = pd.to_numeric(df[cols[i]])

In [183]:
df = df.sort_values(by=['player','season']).reset_index().drop('index',axis=1)

In [225]:
df.head()

,player,age,team_id,lg_id,per,season,g,gs,mp_per_g,fg_per_g,fga_per_g,fg2_per_g,fg2a_per_g,fg3_per_g,fg3a_per_g,ft_per_g,fta_per_g,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,fg_pct,fg2_pct,fg3_pct,efg_pct,ft_pct,ts_pct
0,A.J. Hammons,24,DAL,NBA,8.4,2016-17,22,0,7.4,0.8,1.9,0.5,1.5,0.2,0.5,0.4,0.9,0.4,1.3,1.6,0.2,0.0,0.6,0.5,1.0,2.2,0.405,0.375,0.500,0.464,0.450,0.472
1,A.J. Price,28,TOT,NBA,12.0,2014-15,26,0,12.5,2.0,5.3,1.4,3.1,0.6,2.2,0.6,0.9,0.2,1.0,1.2,1.8,0.3,0.0,0.5,0.6,5.1,0.372,0.450,0.263,0.427,0.667,0.451
2,Aaron Brooks,30,CHI,NBA,14.4,2014-15,82,21,23.0,4.2,10.0,2.7,6.1,1.5,3.8,1.8,2.1,0.4,1.6,2.0,3.2,0.7,0.2,1.9,2.3,11.6,0.421,0.442,0.387,0.495,0.833,0.534
3,Aaron Brooks,31,CHI,NBA,11.8,2015-16,69,0,16.1,2.7,6.8,1.8,4.1,1.0,2.7,0.7,0.9,0.3,1.2,1.5,2.6,0.4,0.1,1.2,1.9,7.1,0.401,0.430,0.357,0.471,0.766,0.494
4,Aaron Brooks,32,IND,NBA,9.5,2016-17,65,0,13.8,1.9,4.6,1.1,2.6,0.7,2.0,0.5,0.6,0.3,0.8,1.1,1.9,0.4,0.1,1.0,1.4,5.0,0.403,0.424,0.375,0.483,0.800,0.507


In [429]:
# saving nba df as json file
df.to_json('nba_df.json')

## Rookies dataframe

In [417]:
rookies_soup = getAndParseUrl('https://www.nba.com/article/2019/06/21/2019-nba-draft-results-picks-1-60')

rookie_names = [r.text for r in rookies_soup.select('div p a[href]') if r.text not in ['officially traded', 'first being dealt', 'then to']]

rookie_names = [r for r in rookie_names if 'traded' not in r]

rookie_names = [r for r in rookie_names if 'officially' not in r][2:-3]

In [418]:
rookie_names

['Zion Williamson',
 'Ja Morant',
 'RJ Barrett',
 "De'Andre Hunter",
 'Darius Garland',
 'Jarrett Culver',
 'Coby White',
 'Jaxson Hayes',
 'Rui Hachimura',
 'Cam Reddish',
 'Cameron Johnson',
 'PJ Washington',
 'Tyler Herro',
 'Romeo Langford',
 'Sekou Doumbouya',
 'Chuma Okeke',
 'Nickeil Alexander-Walker',
 'Goga Bitadze',
 'Luka Samanic',
 'Matisse Thybulle',
 'Brandon Clarke',
 'Grant Williams',
 'Darius Bazley',
 'Ty Jerome',
 'Nassir Little',
 'Dylan Windler',
 'Mfiondu Kabengele',
 'Jordan Poole',
 'Keldon Johnson',
 'Kevin Porter Jr.',
 'Nicolas Claxton',
 'KZ Okpala',
 'Carsen Edwards',
 'Bruno Fernando',
 'Marcos Louzada Silva',
 'Cody Martin',
 'Deividas Sirvydis',
 'Daniel Gafford',
 'Alen Smailagic',
 'Justin James',
 'Eric Paschall',
 'Admiral Schofield',
 'Jaylen Nowell',
 'Bol Bol',
 'Isaiah Roby',
 'Talen Horton-Tucker',
 'Ignas Brazdeikis',
 'Terance Mann',
 'Quinndary Weatherspoon',
 'Jarrell Brantley',
 'Tremont Waters',
 'Jalen McDaniels',
 'Justin Wright-Foreman'

rookie schools

In [277]:
rook_school_soup = getAndParseUrl('https://en.wikipedia.org/wiki/2019_NBA_draft')

In [278]:
# clean the school names 
rookie_schools = list(set([r.text.strip('\n').replace(' (Fr.)','').replace(' (So.)','').replace(' (Jr.)','').replace(' (Sr.)','').lower().replace(' ','-') for r in rook_school_soup.select('tr td:last-child')][11:71]))


In [279]:
rookie_schools

['rytas-vilnius-(lithuania)',
 'georgia',
 'kentucky',
 'hofstra',
 'sesi/franca-(brazil)',
 'belmont',
 'nebraska',
 'indiana',
 'ucla',
 'gonzaga',
 'duke',
 'vanderbilt',
 'miami',
 'arkansas',
 'usc',
 'stanford',
 'villanova',
 'oregon',
 'wyoming',
 'san-diego-state',
 'partizan-belgrade-(serbia)',
 'mississippi-state',
 'iowa-state',
 'texas-tech',
 'mega-bemax-(serbia)',
 'north-carolina',
 'murray-state',
 'limoges-csp-(france)',
 'florida-state',
 'charleston',
 'olimpija-ljubljana-(slovenia)',
 'yale',
 'virginia',
 'nevada',
 'michigan',
 'maryland',
 'virginia-tech',
 'lsu',
 'santa-cruz-warriors-(g-league)',
 'purdue',
 'auburn',
 'texas',
 'princeton-hs-(sharonville,-ohio;-hs-sr.)',
 'washington',
 'tennessee']

scrape for college stats on upcoming rookies

In [324]:
def get_college_players(): # function to grab every player and his stats from the page as a dictionary, and store as list
    
    # player dictionaries container
    players = []
    
    # loop through each school page to grab each player at each school
    for school in tqdm(rookie_schools):
        try:
            url = f'https://www.sports-reference.com/cbb/schools/{school}/2019.html'
            soup = getAndParseUrl(url)

            # select each td tag under tr tag in the per_game table: each td tag in player_soup is a player's stat, all stored as a list
            player_soup = soup.select('#per_game tr td')

            # create slicing points to group the 26 elements(html tags) into single list(player)
            j=0
            k=25

            # len(player_soup) gives the number of stats on each page
            # divide that by 26 to get number of players per page
            for i in range(int(len(player_soup)/25)):

                # each player container (player with stats)
                each_player = {}

                # p is a single player with stats sliced from player_soup
                p = player_soup[j:k]

                #loop through p to create dictionary of each player
                for stat in p:
                    # set each stat equal to the tag text (i.e. pts_per_g = 15.4)
                    each_player[stat.attrs['data-stat']] = stat.text

                # add each player dictionary to the players container
                players.append(each_player)

                # move slicer to next player
                j+=25
                k+=25
        except:
            continue

    return players

In [325]:
rookies = get_college_players()

100%|██████████| 45/45 [01:06<00:00,  1.57s/it]


In [369]:
# set up the rookies dataframe to have the same order of columns as the nba players dataframe
# college stats are missing some columns that nba dataframe has
rookie_df = pd.DataFrame(rookies, columns=[x for x in list(df.columns) if x not in ['efg_pct','ts_pct','age','team_id','lg_id','per','season']])

In [370]:
# change numerical data from strings to ints/floats
for col in rookie_df.columns:
    try:
        rookie_df[col] = pd.to_numeric(rookie_df[col])
    except:
        pass # this is to skip over strings that are not numerical

In [427]:
# filter out all college players that weren't drafted into nba in 2019
#!!!!! ISSUE: MISSING ROOKIE DATA FOR THE ROOKIES NOT DRAFTED FROM COLLEGE (i.e. overseas, G-league, etc.) !!!!!
rookie_df = rookie_df.loc[rookie_df['player'].isin(rookie_names)].sort_values(by='player').reset_index().drop('index',axis=1)

In [434]:
rookie_df.head(1)

,player,g,gs,mp_per_g,fg_per_g,fga_per_g,fg2_per_g,fg2a_per_g,fg3_per_g,fg3a_per_g,ft_per_g,fta_per_g,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,fg_pct,fg2_pct,fg3_pct,ft_pct,efg_pct
0,Admiral Schofield,37,37,31.8,6.4,13.6,4.4,8.8,2.0,4.8,1.6,2.3,1.2,5.0,6.1,2.0,0.9,0.5,1.9,2.5,16.5,0.474,0.505,0.418,0.698,0.544118


In [430]:
# saving rookie df as json file
rookie_df.to_json('rookie_df.json')

##### Adding missing columns to rookie_df to match the nba df
some columns are calculable

missing: ['efg_pct','ts_pct','age','team_id','lg_id','per','season']

In [432]:
# calculating effective field goal percentage for college players (efg_pct)
rookie_df['efg_pct'] = (rookie_df.fg2_per_g + (1.5*rookie_df.fg3_per_g))/rookie_df.fga_per_g

In [439]:
# calculating true shooting percentage (ts_pct)
rookie_df['ts_pct'] = rookie_df.pts_per_g/(2*(rookie_df.fga_per_g + (.44*rookie_df.fta_per_g)))

In [440]:
# calculating player efficiency rating (per) in accordance with basketball-reference.com's formula
# this is the url for calculating per: 'https://www.basketball-reference.com/about/per.html'
factor = (2/3) - (.5*())